In [2]:
!pip install pandas

In [94]:
import pandas as pd
import json
import numpy as np
import math

pd.set_option('display.max_columns', None)

In [107]:
# loading in the JSON data
with open('match_results.json','r') as f:
    match_res = json.load(f)
match_res[0]

{'city': 'Brisbane',
 'dates': '2017-01-13',
 'gender': 'male',
 'match_type': 'ODI',
 'outcome.runs': 92,
 'outcome.winner': 'Australia',
 'overs': 50,
 'player_of_match': 'MS Wade',
 'teams': 'Australia',
 'toss.decision': 'bat',
 'toss.winner': 'Australia',
 'umpires': 'MD Martell',
 'venue': 'Brisbane Cricket Ground, Woolloongabba',
 'matchid': '1000887'}

In [129]:
match_df = pd.DataFrame(match_res)
print("Complete Match Results JSON Shape ", match_df.shape)
match_df = match_df[match_df['gender']=='male']
print("Complete Match Results JSON Shape for Men's matches ", match_df.shape)
match_df = match_df.iloc[:,:21] # dropping columns containing supersub info since not relevant
match_df.head()

Complete Match Results JSON Shape  (4306, 42)
Complete Match Results JSON Shape for Men's matches  (3810, 42)


,city,dates,gender,match_type,outcome.runs,outcome.winner,overs,player_of_match,teams,toss.decision,toss.winner,umpires,venue,matchid,outcome.wickets,result,outcome.method,neutral_venue,match_type_number,outcome.result,outcome.eliminator
0,Brisbane,2017-01-13,male,ODI,92.0,Australia,50,MS Wade,Australia,bat,Australia,MD Martell,"Brisbane Cricket Ground, Woolloongabba",1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Brisbane,2017-01-13,male,ODI,92.0,Australia,50,MS Wade,Pakistan,bat,Australia,C Shamshuddin,"Brisbane Cricket Ground, Woolloongabba",1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2017-01-15,male,ODI,NaN,Pakistan,50,Mohammad Hafeez,Australia,bat,Australia,CB Gaffaney,Melbourne Cricket Ground,1000889,6.0,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,2017-01-15,male,ODI,NaN,Pakistan,50,Mohammad Hafeez,Pakistan,bat,Australia,P Wilson,Melbourne Cricket Ground,1000889,6.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Perth,2017-01-19,male,ODI,NaN,Australia,50,SPD Smith,Australia,field,Australia,SD Fry,Western Australia Cricket Association Ground,1000891,7.0,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
match_df.isna().sum()

city                   622
dates                    0
gender                   0
match_type               0
outcome.runs          2024
outcome.winner         190
overs                    0
player_of_match        248
teams                    0
toss.decision            0
toss.winner              0
umpires                  0
venue                    0
matchid                  0
outcome.wickets       1976
result                3630
outcome.method        3466
neutral_venue         3180
match_type_number     2750
outcome.result        3800
outcome.eliminator    3806
dtype: int64

In [137]:
match_df['result'].value_counts()

result
no result    152
tie           28
Name: count, dtype: int64

In [149]:
# Filter matches: Men’s ODI with a definitive winner
filtered_matches = match_df[
    (match_df['result'] != 'no result')
]

valid_match_ids = filtered_matches['matchid'].unique().tolist()

In [161]:
# Load innings data
with open('innings_results.json', 'r') as f:
    innings_res = json.load(f)

innings_df = pd.DataFrame(innings_res)
# Filter by valid matches
innings_df = innings_df[innings_df['matchid'].isin(valid_match_ids)]
# keeping select columns
select_cols = ['runs.batsman','runs.extras','runs.total','over','team','innings','matchid','wides','wicket.kind','wicket.player_out','wicket.fielders','legbyes','noballs','byes']
innings_df = innings_df.loc[:,select_cols]
print("Dimensions after only keeping relevant matchids and cleaning columns", innings_df.shape)
innings_df.head()

Dimensions after only keeping relevant matchids and cleaning columns (1000742, 14)


,runs.batsman,runs.extras,runs.total,over,team,innings,matchid,wides,wicket.kind,wicket.player_out,wicket.fielders,legbyes,noballs,byes
0,0,0,0,0.1,Australia,1,1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,0,0.2,Australia,1,1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,0.3,Australia,1,1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,0,0.4,Australia,1,1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,1,1,0.5,Australia,1,1000887,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
# Convert over to numeric for sorting and calculations
innings_df['over_numeric'] = innings_df['over'].astype(float)
innings_df = innings_df.sort_values(by=['matchid', 'innings', 'over_numeric']).reset_index(drop=True)
innings_df.head()

,runs.batsman,runs.extras,runs.total,over,team,innings,matchid,wides,wicket.kind,wicket.player_out,wicket.fielders,legbyes,noballs,byes,over_numeric
0,0,0,0,0.1,Australia,1,1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1
1,0,0,0,0.2,Australia,1,1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2
2,0,0,0,0.3,Australia,1,1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3
3,0,0,0,0.4,Australia,1,1000887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4
4,0,1,1,0.5,Australia,1,1000887,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.5


In [163]:
innings_df.isna().sum()

runs.batsman              0
runs.extras               0
runs.total                0
over                      0
team                      0
innings                   0
matchid                   0
wides                978067
wicket.kind          972969
wicket.player_out    972969
wicket.fielders      982227
legbyes              990417
noballs              996855
byes                 999290
over_numeric              0
dtype: int64

In [164]:
innings_df['wicket.kind'].value_counts()

wicket.kind
caught                   15463
bowled                    4891
lbw                       2952
run out                   2882
caught and bowled          806
stumped                    711
retired hurt                38
hit wicket                  24
obstructing the field        6
Name: count, dtype: int64

In [165]:
# checking retired hurt values
hurt_index = innings_df[innings_df['wicket.kind']=='retired hurt'].index[0]
innings_df[hurt_index-2:hurt_index+2]

,runs.batsman,runs.extras,runs.total,over,team,innings,matchid,wides,wicket.kind,wicket.player_out,wicket.fielders,legbyes,noballs,byes,over_numeric
2811,2,0,2,30.4,Pakistan,2,1000895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.4
2812,1,0,1,30.5,Pakistan,2,1000895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.5
2813,0,0,0,30.6,Pakistan,2,1000895,NaN,retired hurt,Shoaib Malik,NaN,NaN,NaN,NaN,30.6
2814,0,0,0,30.7,Pakistan,2,1000895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.7


In [201]:
def process_innings_data(df):
    wickets_fallen = 0
    deliveries_processed = []
    wicket_tags = ['caught', 'bowled', 'lbw', 'run out', 'caught and bowled', 'stumped', 'hit wicket', 'obstructing the field']
    total_overs = 50
    total_balls = total_overs * 6
    legal_deliveries_bowled = 0
    
    for i, row in df.iterrows():
        runs_batsman = row.get('runs.batsman', 0)
        runs_extras = row.get('runs.extras', 0)
        runs_on_ball = runs_batsman + runs_extras

        wicket_on_ball = 1 if (row['wicket.kind'] in wicket_tags) else 0
        wickets_remaining = 10 - wickets_fallen

        # Determine if this ball is legal
        # If it's a wide or no-ball, do not increment legal_deliveries_bowled
        legal = True if (math.isnan(row['wides']) & math.isnan(row['noballs'])) else False
        if legal:
            legal_deliveries_bowled += 1

        # Calculate remaining overs in standard cricket format (O.B)
        balls_remaining = total_balls - legal_deliveries_bowled
        overs_remaining_int = balls_remaining // 6
        balls_remaining_in_over = balls_remaining % 6
        remaining_overs = overs_remaining_int + (balls_remaining_in_over * 0.1)

        deliveries_processed.append({
            'matchid': row['matchid'],
            'team': row['team'],
            'innings': row['innings'],
            'runs_on_ball': runs_on_ball,
            'wicket_on_ball': wicket_on_ball,
            'remaining_overs': remaining_overs,
            'remaining_wickets': wickets_remaining,
            'over_numeric': row['over_numeric']
        })

        if wicket_on_ball:
            wickets_fallen += 1

    return pd.DataFrame(deliveries_processed)

# Apply to each innings
final_balls_df = innings_df[:70].groupby(['matchid', 'innings'], group_keys=False).apply(process_innings_data).reset_index(drop=True)

C:\Users\siddh\AppData\Local\Temp\ipykernel_53140\2421186980.py:46: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_balls_df = innings_df[:70].groupby(['matchid', 'innings'], group_keys=False).apply(process_innings_data).reset_index(drop=True)


In [203]:
final_balls_df.head()

,matchid,team,innings,runs_on_ball,wicket_on_ball,remaining_overs,remaining_wickets,over_numeric
0,1000887,Australia,1,0,0,49.5,10,0.1
1,1000887,Australia,1,0,0,49.4,10,0.2
2,1000887,Australia,1,0,0,49.3,10,0.3
3,1000887,Australia,1,0,0,49.2,10,0.4
4,1000887,Australia,1,1,0,49.2,10,0.5


In [205]:
# Assuming final_balls_df is the dataset you currently have
# Extract the integer over number from over_numeric
final_balls_df['over_number'] = final_balls_df['over_numeric'].apply(lambda x: int(str(x).split('.')[0]))

# Get sum of runs per over
over_runs_df = final_balls_df.groupby(['matchid', 'innings', 'over_number'])['runs_on_ball'].sum().reset_index(name='runs_this_over')

# Get the state at the start of the over from the first legal delivery of that over
over_state_df = final_balls_df.groupby(['matchid', 'innings', 'over_number']).first().reset_index()
over_state_df['remaining_overs'] = 49 - over_state_df['over_number']
over_data = over_state_df[['matchid', 'innings', 'over_number','remaining_overs', 'team', 'remaining_wickets']].merge(
    over_runs_df, on=['matchid', 'innings', 'over_number'], how='inner'
)

# Now over_data contains one row per over with:
# matchid, innings, over_number, team, remaining_overs, remaining_wickets, runs_this_over
over_data.head(10)

,matchid,innings,over_number,remaining_overs,team,remaining_wickets,runs_this_over
0,1000887,1,0,49,Australia,10,1
1,1000887,1,1,48,Australia,10,1
2,1000887,1,2,47,Australia,10,3
3,1000887,1,3,46,Australia,10,6
4,1000887,1,4,45,Australia,10,2
5,1000887,1,5,44,Australia,8,8
6,1000887,1,6,43,Australia,8,4
7,1000887,1,7,42,Australia,8,13
8,1000887,1,8,41,Australia,8,14
9,1000887,1,9,40,Australia,8,9
